# Projeto 1 - Ciência dos Dados

Nome: Lorena Barbosa Antunes Da Silva

Nome: José Edson Mendonça Ribeiro Lima Araújo

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import emoji

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\José Edson Mendonça\OneDrive - Insper - Institudo de Ensino e Pesquisa\Documentos\2° Semestre\CDADOS\CDados_Projeto1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'Peaky Blinders.xlsx'

In [4]:
train = pd.read_excel(filename)
linha=np.arange(0,300,1)
train=train.iloc[linha,[0,1]]
train.head(5)
#train.tail()

,Treinamento,Classificação
0,peaky fucking blinders foi a melhor coisa que ...,1.0
1,@limawt1 peaky blinders está vindo em breve!! ...,0.0
2,gente cancela ele me seguiu de outro instagram...,0.0
3,até eu que não sou fumante fico com vontade de...,0.0
4,tou a curtir bue de peaky blinders,1.0


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
linha=np.arange(0,200,1)
test=test.iloc[linha,[0,1]]
test.head(5)
test.tail(5)

,Teste,Classificação
195,viciada em peaky blinders 🖤,1.0
196,@pedropossobom peaky blinders mastermind \nmei...,1.0
197,@italoverdose peaky blinders está vindo em bre...,0.0
198,"a gente fuma tbem, somos os peaky blinders htt...",0.0
199,o lado bom de tá com covid e que eu tô maraton...,1.0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

Produto: Série Peaky Blinders, consideramos relevante qualquer comentário positivo ou negativo, relacionados a série mostrando que estão assistindo ou indicando. Exemplo: "Não curti assistir Peaky Blinders", "@user1 vc precisa assistir peaky blinders", "acabei de maratonar essa serie peaky blinders".

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
from IPython.display import display
pd.options.display.max_rows = 13


import re 



def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    limpa_emoji=' '.join(emoji.get_emoji_regexp().split(text))
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', limpa_emoji)
    
    return text_subbed

In [12]:
comentarios_relevantes=[]
comentarios_irrelevantes=[]
for i in range(0,300,1):
    if train.Classificação[i]==1:
        comentario=cleanup(train.Treinamento[i]).lower().split()
        comentarios_relevantes.append(comentario)
    elif train.Classificação[i]==0:
        comentario=cleanup(train.Treinamento[i]).lower().split()
        comentarios_irrelevantes.append(comentario)

comentarios_relevantes

[['peaky',
  'fucking',
  'blinders',
  'foi',
  'a',
  'melhor',
  'coisa',
  'que',
  'comecei',
  'a',
  'ver'],
 ['tou', 'a', 'curtir', 'bue', 'de', 'peaky', 'blinders'],
 ['@kika_cavalcante',
  'poxa',
  'eu',
  'curtia',
  'filmes',
  'mas',
  'ultimamente',
  'é',
  'tanta',
  'lacração',
  'que',
  'ando',
  'sem',
  'vontade',
  'a',
  'última',
  'série',
  'que',
  'vi',
  'e',
  'até',
  'gostei',
  'foi',
  'peaky',
  'blinders'],
 ['eu',
  'nao',
  'entendo',
  'nada',
  'de',
  'peaky',
  'blinders',
  'to',
  'na',
  'terceira',
  'temp',
  'e',
  'ainda',
  'fico',
  'de',
  'onde',
  'surgiu',
  'esse',
  'homem'],
 ['viciadíssima', 'em', 'peaky', 'blinders'],
 ['assistir', 'peaky', 'blinders', 'essa', 'madrugada', 'até', 'dormir'],
 ['já',
  'eh',
  'a',
  'quarta',
  'vez',
  'q',
  'eu',
  'tô',
  'assistindo',
  'peaky',
  'blinders',
  'e',
  'parece',
  'q',
  'fica',
  'melhor',
  'cada',
  'vez'],
 ['enfim', 'comecei', 'peaky', 'blinders'],
 ['quero', 'a', 'no

In [18]:
lista_remocao=[]
for comentario in comentarios_irrelevantes:
    for palavra in comentario:
        if  palavra[0]=='@' or 'htt' == palavra[0:3]:
            lista_remocao.append(palavra)
        if palavra in lista_remocao:
            comentario.remove(palavra)
        
    
comentarios_irrelevantes


[['peaky',
  'blinders',
  'está',
  'vindo',
  'em',
  'breve',
  'o',
  'ator',
  'da',
  '6ª',
  'temporada',
  'da',
  'série',
  'revelou',
  'uma',
  'possível',
  'época',
  'para',
  'a',
  'estreia',
  'confira',
  'em',
  'grata',
  'sua',
  'ajuda',
  'é',
  'importante',
  'demais',
  'ajude',
  'empresas',
  'pequenas',
  'elas',
  'agradecem',
  'muito',
  '💛'],
 ['gente',
  'cancela',
  'ele',
  'me',
  'seguiu',
  'de',
  'outro',
  'instagram',
  'para',
  'a',
  'namorada',
  'dele',
  'não',
  'ver',
  'as',
  'mensagens',
  'e',
  'ele',
  'é',
  'fã',
  'de',
  'peaky',
  'blinders',
  '🗣',
  '️',
  '🗣',
  '️',
  '🗣',
  '️',
  '🗣',
  '️',
  '🗣',
  '️',
  '🗣',
  '️',
  '🗣',
  '️'],
 ['até',
  'eu',
  'que',
  'não',
  'sou',
  'fumante',
  'fico',
  'com',
  'vontade',
  'de',
  'comprar',
  'um',
  'malborozinho',
  'quando',
  'assisto',
  'peaky',
  'blinders'],
 ['ninguém',
  'me',
  'decepciona',
  'porque',
  'eu',
  'pertenço',
  'á',
  'gangue',
  'dos',
  '

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**